# Understanding sequence prediction 

Sequence prediction is about guessing the next step in a series. It uses what happened before. It is key in making guesses right in many areas like guessing what you want to see on the web, what you might buy next, or what the weather will be.

There are several ways to guess in sequence prediction. Each way has its good points. Some use Markov models. These guess by looking at the last step. They are easy and good at guessing.

Markov models: These models guess the next step by looking at the current step only. They are simple and do their job well.

Directed graphs: These show how steps relate. They draw a picture of what comes next based on what happened before. This helps see how things are connected.

Recurrent neural networks (RNNs): RNNs are good at remembering what happened in a series step by step. They can spot long patterns well, so they make good guesses about the future.
Each way to guess has its own good and bad points. Picking the right way depends on what you need to think. In the next part, we will talk more about these ways. We will see how they work in different areas.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import bokeh as bk
import json

In [2]:
x_train = pd.read_csv(r"C:\Users\PC\Desktop\HRFlow - Recommandation d'emploi basée sur le comportement\x_train_Meacfjr.csv")
y_train = pd.read_csv(r"C:\Users\PC\Desktop\HRFlow - Recommandation d'emploi basée sur le comportement\y_train_SwJNMSu.csv")

display(x_train.head())
display(y_train.head())

,session_id,job_ids,actions
0,0,"[305, 299, 300, 290, 282, 274, 264, 261]","['view', 'view', 'view', 'view', 'view', 'view..."
1,1,"[84, 257, 252, 250]","['view', 'view', 'view', 'view']"
2,2,"[241, 237, 221, 309, 310, 306, 301]","['view', 'view', 'apply', 'apply', 'apply', 'a..."
3,3,"[303, 297, 296, 298, 294, 295, 292, 293]","['apply', 'apply', 'apply', 'apply', 'apply', ..."
4,4,"[171, 291, 289, 166, 288, 155]","['apply', 'apply', 'apply', 'apply', 'apply', ..."


,session_id,job_id,action
0,0,84,view
1,1,241,view
2,2,303,apply
3,3,171,apply
4,4,286,apply


In [3]:
# Get all unique job_ids from the dataset
all_job_ids = set()
job_action_pairs = set()

# Build our sets
for _, row in x_train.iterrows():
    job_ids = row['job_ids']
    actions = row['actions']
    
    # Add all job_ids to the full set
    all_job_ids.update(job_ids)
    
    # Add each job-action pair
    for job_id, action in zip(job_ids, actions):
        job_action_pairs.add((job_id, action))

# Find jobs without actions
jobs_without_actions = all_job_ids - {pair[0] for pair in job_action_pairs}

print(f"Total unique job_ids: {len(all_job_ids)}")
print(f"Number of job_ids missing actions: {len(jobs_without_actions)}")
print("\nJob IDs missing actions:")
print(list(jobs_without_actions))


Total unique job_ids: 14
Number of job_ids missing actions: 0

Job IDs missing actions:
[]


In [4]:
with open('job_listings.json') as f:
    data = json.load(f)

emploi = pd.json_normalize(data)

In [5]:
emploi.head()

,0,1,2,3,4,5,6,7,8,9,...,27359,27360,27361,27362,27363,27364,27365,27366,27367,27368
0,TITLE\nQA Intégration / Data Analyst - SalesF...,TITLE\nIngénieur Système\n\nSUMMARY\nNous re...,TITLE\nTesteur QA Automatisation Cypress\n\nSU...,TITLE\nIngénieur support N3 IP - PARIS \n\nSU...,TITLE\nBusiness Analyst MOA FRONT\n\nSUMMARY\n...,TITLE\nBusiness Analyst SAP S/4\n\nSUMMARY\nNo...,TITLE\nSalesforce Marketing Cloud Product Owne...,TITLE\nResponsable Sécurité Opérationnel \n...,TITLE\nArchitecte d'entreprise Data BI\n\nSUMM...,TITLE\nIngénieur.e QA web / mobile - (Haute-S...,...,TITLE\nParis-Administrateur Système DB2 - Z/O...,TITLE\nIngénieur d’intégration applicative\n...,"TITLE\nConsultant MS BI Confirmé Build & Run,...",TITLE\nData modeler / Senior\n\nSUMMARY\nNous ...,TITLE\nLyon-Développeur Cobol Mainframe-R2030...,TITLE\nIncident Manager e-commerce\n\nSUMMARY\...,TITLE\nConsultant Azure Security\n\nSUMMARY\nK...,TITLE\nChef de projet Supply Chain\n\nSUMMARY\...,TITLE\nPO Infrastructure\n\nSUMMARY\nUn Produc...,"TITLE\nData Engineer Senior Spark, Scala, Data..."


In [6]:
emploi = emploi.transpose()

In [7]:
emploi

,0
0,TITLE\nQA Intégration / Data Analyst - SalesF...
1,TITLE\nIngénieur Système\n\nSUMMARY\nNous re...
2,TITLE\nTesteur QA Automatisation Cypress\n\nSU...
3,TITLE\nIngénieur support N3 IP - PARIS \n\nSU...
4,TITLE\nBusiness Analyst MOA FRONT\n\nSUMMARY\n...
...,...
27364,TITLE\nIncident Manager e-commerce\n\nSUMMARY\...
27365,TITLE\nConsultant Azure Security\n\nSUMMARY\nK...
27366,TITLE\nChef de projet Supply Chain\n\nSUMMARY\...
27367,TITLE\nPO Infrastructure\n\nSUMMARY\nUn Produc...


In [8]:
emploi['job_id'] = emploi.index

emploi

,0,job_id
0,TITLE\nQA Intégration / Data Analyst - SalesF...,0
1,TITLE\nIngénieur Système\n\nSUMMARY\nNous re...,1
2,TITLE\nTesteur QA Automatisation Cypress\n\nSU...,2
3,TITLE\nIngénieur support N3 IP - PARIS \n\nSU...,3
4,TITLE\nBusiness Analyst MOA FRONT\n\nSUMMARY\n...,4
...,...,...
27364,TITLE\nIncident Manager e-commerce\n\nSUMMARY\...,27364
27365,TITLE\nConsultant Azure Security\n\nSUMMARY\nK...,27365
27366,TITLE\nChef de projet Supply Chain\n\nSUMMARY\...,27366
27367,TITLE\nPO Infrastructure\n\nSUMMARY\nUn Produc...,27367


In [22]:
# Rename column 0 to job_desciption
emploi.rename(columns={0: 'job_description'}, inplace=True)

In [24]:
def extract_job_sections(df, text_column='description'):
    """
    Extract job sections from text descriptions based on flags like TITLE, SUMMARY, etc.
    
    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame containing the job descriptions
    text_column : str, default='description'
        The column containing the job description text
        
    Returns:
    --------
    pandas DataFrame
        Original DataFrame with added columns for job_title and summary
    """
    import re
    
    # Create copies of columns to avoid modifying the original DataFrame
    df = df.copy()
    
    # Initialize new columns
    df['job_title'] = ''
    df['summary'] = ''
    
    for idx, row in df.iterrows():
        text = row[text_column]
        
        # Extract job title (between TITLE and SUMMARY)
        title_match = re.search(r'TITLE\s*\n(.*?)(?=\s*\n\s*SUMMARY)', text, re.DOTALL)
        if title_match:
            df.at[idx, 'job_title'] = title_match.group(1).strip()
        
        # Extract summary (after SUMMARY and before any other section flag)
        summary_match = re.search(r'SUMMARY\s*\n(.*?)(?=\s*\n\s*SECTION|\Z)', text, re.DOTALL)
        if summary_match:
            df.at[idx, 'summary'] = summary_match.group(1).strip()
    
    return df


In [25]:

processed_emploi = extract_job_sections(emploi, text_column='job_description')

# Display the first few rows to check the results
processed_emploi[['job_title', 'summary']].head()


,job_title,summary
0,QA Intégration / Data Analyst - SalesForces Sales Cloud,"Responsabilités :\nAssurer la qualité des données intégrées entre Salesforce et d'autres systèmes (SAP, autres produits).\nDévelopper et exécuter des plans de tests pour les fonctionnalités Salesforce Sales Cloud.\nIdentifier, documenter et suivre les anomalies jusqu'à leur résolution.\nCollaborer avec les équipes de développement pour comprendre les besoins et les spécifications.\nEffectuer des analyses de données pour soutenir les décisions métier et améliorer les processus.\nParticiper à la qualification fonctionnelle des solutions en évolution.\nContribuer à la documentation des processus et des configurations"
1,Ingénieur Système,"Nous recherchons un Ingénieur Système pour notre client. Le candidat idéal devra posséder des compétences variées en environnement système, virtualisation, Cloud et DevOps, et être capable de travailler dans des environnements hybrides et publics.\nCe poste offre une opportunité de contribuer activement à la gestion des infrastructures et à l'automatisation des processus au sein d'une équipe dynamique.\nGestion des Environnements Système :\nAdministrer et maintenir des systèmes Linux et Windows.\nGérer des environnements de virtualisation (VMware, hyperviseurs).\nCloud et Virtualisation :\nTravailler avec des environnements Cloud Public (Azure, GCP).\nAssurer l’intégration et la gestion des ressources Cloud dans des environnements hybrides.\nMiddleware et Bases de Données :\nAdministrer des serveurs middleware (Apache, Tomcat).\nGérer des bases de données SQL Server et assurer leur performance et leur sécurité.\nDevOps et Automatisation :\nMettre en œuvre des p..."
2,Testeur QA Automatisation Cypress,"Vous avez au moins une première expérience sur l’automatisation de tests\nLa connaissance de Jenkins et la certification ISTQB est un plus\nMissions :\nParticiper à la définition d'une stratégie d'automatisation des tests\nDévelopper les scripts\nExécuter les scripts d'automatisations\nMettre en œuvre des jeux de données\nAssurer l'analyse et la résolution des problèmes détectés lors de l'exécution des tests automatiques.\nAssister les utilisateurs\nAssurer la rédaction des rapports de tests et de toute documentation technique afférente au périmètre.\nStack technique\nTests :\nJest\nCypress\nJmeter\nJavascript, React"
3,Ingénieur support N3 IP - PARIS,"Dans le cadre de cette mission :\nVous garantissez le support technique N3 afin d'accroître la fiabilité, la performance et la qualité de service du réseau d'une importante entreprise française à Paris .\nVos missions principales seront les suivantes :\nApporter un support technique sur le fonctionnement des équipements\nContribuer à l'intégration et au maintien en condition opérationnelle des équipements. (MCO)\nIdentifier ou analyser les dysfonctionnements probables ou rencontrés\nEffectuer des vérifications régulières de bon fonctionnement\nContribuer à l'optimisation de l'utilisation des équipements\nAssurer le transfert de compétences aux équipes, participer à leur formation."
4,Business Analyst MOA FRONT,"Nous recherchons un (e) consultant(e) ayant une expérience de minimum 4 ans en MOA et gestion de projet.\nLe consultant(e) devra faire preuve d’autonomie, de rigueur, de méthode, d’une faculté à travailler en équipe, d’une bonne aisance relationnelle ainsi que de bonnes capacités rédactionnelles et d’un esprit de synthèse.\nLe consultant(e) aura pour rôle de prendre en charge des projets portant sur les applications CRM et outils maison.\nDans ce cadre, le Prestataire aura pour missions :\nL'analyse et le cadrage des besoins auprès des équipes Métiers\nLa planification et le chiffrage des projets en liaison avec les responsables Etudes\nLa participation à la rédaction des livrables projets (notes d'études, spécifications fonctionnelles, recette, PV).\nL’accompagnement d

In [27]:
processed_emploi.drop(columns=["job_description"], inplace=True)

In [28]:
processed_emploi.head(2)

,job_id,job_title,summary
0,0,QA Intégration / Data Analyst - SalesForces Sales Cloud,"Responsabilités :\nAssurer la qualité des données intégrées entre Salesforce et d'autres systèmes (SAP, autres produits).\nDévelopper et exécuter des plans de tests pour les fonctionnalités Salesforce Sales Cloud.\nIdentifier, documenter et suivre les anomalies jusqu'à leur résolution.\nCollaborer avec les équipes de développement pour comprendre les besoins et les spécifications.\nEffectuer des analyses de données pour soutenir les décisions métier et améliorer les processus.\nParticiper à la qualification fonctionnelle des solutions en évolution.\nContribuer à la documentation des processus et des configurations"
1,1,Ingénieur Système,"Nous recherchons un Ingénieur Système pour notre client. Le candidat idéal devra posséder des compétences variées en environnement système, virtualisation, Cloud et DevOps, et être capable de travailler dans des environnements hybrides et publics.\nCe poste offre une opportunité de contribuer activement à la gestion des infrastructures et à l'automatisation des processus au sein d'une équipe dynamique.\nGestion des Environnements Système :\nAdministrer et maintenir des systèmes Linux et Windows.\nGérer des environnements de virtualisation (VMware, hyperviseurs).\nCloud et Virtualisation :\nTravailler avec des environnements Cloud Public (Azure, GCP).\nAssurer l’intégration et la gestion des ressources Cloud dans des environnements hybrides.\nMiddleware et Bases de Données :\nAdministrer des serveurs middleware (Apache, Tomcat).\nGérer des bases de données SQL Server et assurer leur performance et leur sécurité.\nDevOps et Automatisation :\nMettre en œuvre des p..."


# En fait le projet

En réalité le projet est en deux parties :
- Premièrement être capable de prédire le top 10 jobs par user_ID
- Deuxièmement être capable de prédire l'action sur le premier job de cette liste

# Analyse stat

In [9]:
# Visualize random predictions to understand what we are waiting for 
dum_pred = pd.read_csv(r"C:\Users\PC\Desktop\HRFlow - Recommandation d'emploi basée sur le comportement\random_predictions_iNUaFEi.csv")
dum_pred.head()

,session_id,action,job_id
0,0,apply,"[2932, 21375, 18814, 7610, 17536, 24206, 23696..."
1,1,view,"[5761, 1191, 13009, 25820, 16233, 6106, 24544,..."
2,2,view,"[9778, 18611, 19752, 3517, 10979, 9325, 14889,..."
3,3,view,"[16185, 4398, 16523, 15337, 8941, 6319, 26504,..."
4,4,apply,"[23958, 15038, 21244, 8431, 23548, 6102, 462, ..."


In [10]:
x_train

,session_id,job_ids,actions
0,0,"[305, 299, 300, 290, 282, 274, 264, 261]","['view', 'view', 'view', 'view', 'view', 'view..."
1,1,"[84, 257, 252, 250]","['view', 'view', 'view', 'view']"
2,2,"[241, 237, 221, 309, 310, 306, 301]","['view', 'view', 'apply', 'apply', 'apply', 'a..."
3,3,"[303, 297, 296, 298, 294, 295, 292, 293]","['apply', 'apply', 'apply', 'apply', 'apply', ..."
4,4,"[171, 291, 289, 166, 288, 155]","['apply', 'apply', 'apply', 'apply', 'apply', ..."
...,...,...,...
15877,15877,"[26581, 27314, 27305, 27327, 27138, 27153]","['apply', 'apply', 'apply', 'apply', 'apply', ..."
15878,15878,"[27220, 27219, 27194]","['view', 'view', 'view']"
15879,15879,"[27211, 27210, 27209]","['view', 'view', 'view']"
15880,15880,"[27233, 27220, 27219, 27232, 27231]","['apply', 'view', 'view', 'view', 'view']"
